<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/CDAs/CDA_categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 한글 폰트 설치 및 시각화

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Reuntime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.0 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121730 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset

In [ ]:
import pandas as pd
df_FRUITS = pd.read_csv('fruitstore_saleshistory.csv')
df_FRUITS[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05


## X(범주형) / Y(범주형) 케이스

- 과일 유형(X)과 고객 세그먼트(Y) 사이에 연관성이 있습니까?
 + 분석 방법: 카이제곱 독립성 검정을 사용하여 서로 다른 고객 세그먼트에 걸쳐 과일 유형의 판매 분포가 독립적인지 평가합니다.


In [ ]:
# 고객 세그먼트
df_FRUITS[['연령대', '성별']].value_counts()

연령대    성별
40대    여     196743
50대    여     126952
30대이하  여      72138
60대    여      70064
70대이상  여      40819
40대    남       9793
30대이하  남       8776
50대    남       6756
70대이상  남       4455
60대    남       3755
dtype: int64

In [ ]:
# 과일 유형
fruits = '물품중분류 == "과일"'
df_FRUITS_fruits = df_FRUITS.query(fruits)
df_FRUITS_fruits['물품소분류'].unique()

array(['사과', '건대추', '귤(조생)', '배', '산딸기', '단감', '참다래', '레몬', '블루베리', '곶감',
       '귤만감', '냉동건조과일', '홍시용감(대봉)', '귤(비가림)'], dtype=object)

In [ ]:
df_FRUITS_fruits[['물품중분류', '물품소분류', '성별', '연령대']]

,물품중분류,물품소분류,성별,연령대
0,과일,사과,여,40대
62,과일,건대추,여,50대
132,과일,귤(조생),여,60대
187,과일,귤(조생),여,40대
220,과일,배,여,50대
...,...,...,...,...
539938,과일,사과,여,60대
540103,과일,참다래,여,30대이하
540189,과일,참다래,여,70대이상
540240,과일,산딸기,남,60대


In [ ]:
pd.pivot_table(data=df_FRUITS_fruits[['물품소분류', '성별', '연령대']]
               , index=['연령대'], values=['물품소분류'], aggfunc='count')

,물품소분류
연령대,
30대이하,1579
40대,4357
50대,3422
60대,2068
70대이상,1453


In [ ]:
import numpy as np
pivot_df = pd.pivot_table(df_FRUITS_fruits, index=['물품소분류', '성별'], values='물품명', columns='연령대', aggfunc='count')
pivot_df = pivot_df.fillna(0)
pivot_df

연령대          30대이하     40대     50대    60대  70대이상
물품소분류    성별                                     
건대추      남     5.0     6.0     3.0    5.0    2.0
         여    20.0    66.0    56.0   50.0   38.0
곶감       남     6.0     9.0     2.0    0.0    3.0
         여    32.0   140.0   133.0   66.0   44.0
귤(비가림)   남     3.0     5.0     1.0    0.0    8.0
         여    22.0    81.0    56.0   25.0   26.0
귤(조생)    남    19.0    32.0    11.0    7.0   13.0
         여   149.0   512.0   297.0  113.0   81.0
귤만감      남    20.0    34.0    14.0   14.0   10.0
         여   158.0   531.0   510.0  268.0  155.0
냉동건조과일   남     0.0     0.0     1.0    0.0    0.0
         여     0.0     2.0     0.0    0.0    0.0
단감       남     8.0    23.0     8.0    5.0   12.0
         여    66.0   260.0   198.0  107.0   76.0
레몬       남     1.0     2.0     0.0    2.0    1.0
         여    17.0    51.0    37.0   25.0   15.0
배        남    12.0    29.0    18.0   13.0   14.0
         여   184.0   397.0   264.0  182.0  128.0
블루베리     남     3.0     1.0     3.0    0.0    0.0
         여    19.0    41.0    31.0   14.0    9.0
사과       남    70.0    94.0    76.0   36.0   48.0
         여   535.0  1601.0  1400.0  860.0  568.0
산딸기      남     5.0     3.0     0.0    3.0    1.0
         여    59.0    92.0    65.0   48.0   22.0
참다래      남    24.0    18.0     9.0    8.0   22.0
         여   141.0   317.0   215.0  200.0  152.0
홍시용감(대봉) 남     0.0     0.0     0.0    1.0    0.0
         여     1.0    10.0    14.0   16.0    5.0

In [ ]:
import scipy.stats as stats
stats.chi2_contingency(observed=pivot_df, correction=True)

Chi2ContingencyResult(statistic=478.7357257303993, pvalue=4.0875673882418173e-48, dof=108, expected_freq=array([[2.57465642e+00, 7.10435593e+00, 5.57978104e+00, 3.37200093e+00,
        2.36920568e+00],
       [2.81986179e+01, 7.78096125e+01, 6.11118876e+01, 3.69314388e+01,
        2.59484432e+01],
       [2.45205373e+00, 6.76605327e+00, 5.31407718e+00, 3.21142946e+00,
        2.25638637e+00],
       [5.08801149e+01, 1.40395605e+02, 1.10267101e+02, 6.66371613e+01,
        4.68200171e+01],
       [2.08424567e+00, 5.75114528e+00, 4.51696560e+00, 2.72971504e+00,
        1.91792841e+00],
       [2.57465642e+01, 7.10435593e+01, 5.57978104e+01, 3.37200093e+01,
        2.36920568e+01],
       [1.00534203e+01, 2.77408184e+01, 2.17877164e+01, 1.31668608e+01,
        9.25118410e+00],
       [1.41238295e+02, 3.89724668e+02, 3.06090846e+02, 1.84978337e+02,
        1.29967855e+02],
       [1.12794472e+01, 3.11238450e+01, 2.44447550e+01, 1.47725755e+01,
        1.03793773e+01],
       [1.98861558e+02

In [36]:
round(4.0875673882418173e-48)

0

### 분석 결과
- 통계 결론 : 귀무가설 기각
- 사실 결론 : 고객 세그먼트(연령대, 성별)가 과일 유형과 관계가 있다.
